# 아이펠톤에서 사용할 텍스트 생성 모델 테스트

- 241009 text 생성 다시 테스트
- [survay 논문](https://discuss.pytorch.kr/t/llm-synthetic-data-survey/4764/1) 참고함

## 코드 흐름
- 아래 정보가 적혀 있는 구글 스프레드 시트 가져옴
    - 모델 선택
    - 파라미터 선택
    - 생성할 텍스트 개수 선택
    - 인지 왜곡 문장 선택
    - 인지 왜곡 class 선택
    - 인지 왜곡 몇개 예시로 줄지 선택 
    - 프롬프트 입력
- 벤치마킹 데이터에서 인지 왜곡 class와 같은 데이터 가져옴
- 최종 프롬프트 생성
- text gen
- 구글스프레드 시트에 저장

In [1]:
import pandas as pd
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch
from dotenv import load_dotenv
import os
import time
import gc
import gspread, os
from oauth2client.service_account import ServiceAccountCredentials
import random
import math

c:\Users\saink\py_purpose\hugginhface\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# CUDA가 사용 가능한지 확인하여 device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 출력해서 현재 선택된 device 확인
print(f"Using device: {device}")

# .env 파일에서 환경 변수 로드
load_dotenv()
# .env 파일에서 Hugging Face 토큰 불러오기
token = os.getenv('HUGGINGFACE_TOKEN')

Using device: cuda


In [3]:
# 구글 API 사용을 위한 상수들
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
key_file_name = 'aiffelthon-438107-8b246f7e616c.json' # 아까 받은 json 인증키 파일 경로가 들어가면 됨.

credentials = ServiceAccountCredentials.from_json_keyfile_name(key_file_name, scope)

In [4]:
spreadsheet = gspread.authorize(credentials).open("prompt")
worksheet = spreadsheet.worksheet("test01")
# worksheet = spreadsheet.worksheet("tmp")
resultsheet = spreadsheet.worksheet("test01_re")

# 스프레드시트 데이터를 가져와 데이터프레임으로 변환
data = worksheet.get_all_records()
df = pd.DataFrame(data)

# gen_num = 0 인 것 제거
df = df[df['gen_num'] != 0]

# 데이터프레임의 마지막 3개 row 출력
print(df.tail(3))

                  model_name  max_new_tokens temperature top_p top_k  \
8   Qwen/Qwen2.5-7B-Instruct            1024         0.7          50   
9   Qwen/Qwen2.5-7B-Instruct            1024         0.7          30   
10  Qwen/Qwen2.5-7B-Instruct            1024         0.7          50   

   repetition_penalty  gen_num  example_num  \
8                 1.5        5            3   
9                 1.5        5            1   
10                1.5        5            1   

                                            prompt_01  \
8   You are a famous essayist. I want you to write...   
9   I am a psychologist. I want you to prepare you...   
10  I am a psychologist. I want you to prepare you...   

                                       prompt_02  \
8   I want a <emotion> can be felt in a writing;   
9   I want a <emotion> can be felt in a writing;   
10  I want a {emotion} can be felt in a writing;   

                                            prompt_03  \
8   You should put <sentenc

In [5]:
# 데이터 가져오기
## 파일 이름
raw_filename = 'raw_data/meta0911.csv'
benchmark_filename = 'raw_data/Annotated_data.csv'

In [6]:
# 데이터 로드
meta_data = pd.read_csv(raw_filename, encoding='latin1')
print(meta_data.columns)
print("--------------------------")

benchmark_data = pd.read_csv(benchmark_filename)
print(benchmark_data.columns)

Index(['persona', 'pattern', 'pattern_def', 'thought', 'scenario',
       'persona_in_scenario', 'thought_in_scenario'],
      dtype='object')
--------------------------
Index(['Id_Number', 'Patient Question', 'Distorted part',
       'Dominant Distortion', 'Secondary Distortion (Optional)'],
      dtype='object')


In [7]:
# 데이터 전처리
## meta 데이터의 class 이름이 다른 부분 똑같이 맞추기
meta_data['pattern'] = meta_data['pattern'].replace('Mental filtering', 'Mental filter')
meta_data['pattern'] = meta_data['pattern'].replace('Labeling and mislabeling', 'Labeling')
meta_data['pattern'] = meta_data['pattern'].replace('Jumping to conclusions: mind reading', 'Mind Reading')
meta_data['pattern'] = meta_data['pattern'].replace('Jumping to conclusions: Fortune-telling', 'Fortune-telling')
meta_data['pattern'] = meta_data['pattern'].replace('Catastrophizing', 'Magnification')
meta_data['pattern'] = meta_data['pattern'].replace('Black-and-white or polarized thinking / All or nothing thinking', 'All-or-nothing thinking')

# 개수 확인
print(meta_data['pattern'].value_counts())

pattern
Magnification              200
Fortune-telling            199
Overgeneralization         197
Mind Reading               197
Labeling                   196
All-or-nothing thinking    188
Personalization            185
Mental filter              185
Should statements          179
Name: count, dtype: int64


In [8]:
# benchmark 데이터에서 Secondary Distortion (Optional) 컬럼에 값 있는 것 제외
benchmark_data = benchmark_data[benchmark_data['Secondary Distortion (Optional)'].isnull()]

# 개수 확인
print(benchmark_data['Dominant Distortion'].value_counts())

Dominant Distortion
No Distortion              933
Mind Reading               199
Overgeneralization         187
Magnification              145
Personalization            104
Fortune-telling            102
Labeling                   102
Emotional Reasoning         98
Should statements           86
Mental filter               81
All-or-nothing thinking     77
Name: count, dtype: int64


In [9]:
# 다양성을 주기 위한 감정 리스트
emtion_list = ['Depression', 'anger', 'anxiety', 'disappointment', 'helplessness']

# 인지왜곡 정의 
definition_dict = {
    "Emotional Reasoning": "Believing 'I feel that way, so it must be true'.",
    "Overgeneralization": "Drawing conclusions with limited and often un negative experience.",
    "Mental Filter": "Focusing only on limited negative aspects and not the excessive positive ones.",
    "Should Statements": "Expecting things or personal behavior should be a certain way.",
    "All-or-nothing thinking": "Binary thought pattern. Considering anything short of perfection as a failure.",
    "Mind Reading": "Concluding that others are reacting negatively to you, without any basis in fact.",
    "Fortune-telling": "Predicting that an event will always result in the worst possible outcome.",
    "Magnification": "Exaggerating or Catastrophizing the outcome of certain events or behavior.",
    "Personalization": "Holding oneself personally responsible for events beyond one’s control.",
    "Labeling": "Attaching labels to oneself or others (ex: 'loser', 'perfect')."
}

In [10]:
previouse_model_name = None

for index, row in df.iterrows():
    print(row)
    
    # 모델 로드, 이전과 다른 모델이여야 로드함
    model_name = row['model_name']
    
    if previouse_model_name != model_name:
        try:
            # 모델과 토크나이저 로드 및 pad_token_id 설정
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
            
            # pad_token_id가 설정되지 않았다면 eos_token_id로 설정
            if model.config.pad_token_id is None:
                model.config.pad_token_id = tokenizer.eos_token_id
            
        except Exception as e:
            print(f"Failed to load model: {model_name}. Error: {str(e)}")
            resultsheet.append_row(row.tolist(), table_range='A1') # Google 스프레드시트에 새로운 행 추가
            continue
    
    previouse_model_name = model_name
    
    # --------------------------------------------------- #
    
    # 파라미터 가져오기
    params = {
        'max_new_tokens': int(row['max_new_tokens']) if pd.notna(row['max_new_tokens']) and row['max_new_tokens'] != '' else None,
        'temperature': row['temperature'] if pd.notna(row['temperature']) and row['temperature'] != '' else None,
        'top_p': row['top_p'] if pd.notna(row['top_p']) and row['top_p'] != '' else None,
        'top_k': int(row['top_k']) if pd.notna(row['top_k']) and row['top_k'] != '' else None,
        'repetition_penalty': row['repetition_penalty'] if pd.notna(row['repetition_penalty']) and row['repetition_penalty'] != '' else None
    }

    # None 값을 제거하여 기본값을 사용하게 함
    params = {k: v for k, v in params.items() if v is not None}

    # --------------------------------------------------- #

    # 프롬프트 생성
    # prompt_01부터 prompt_12까지의 컬럼 값을 줄바꿈 형식으로 합치기
    raw_prompt = '\n'.join(row[f'prompt_{i:02d}'] for i in range(1, 13) if df.iloc[0][f'prompt_{i:02d}'])
    prompt = raw_prompt
        
    # 생성할 개수
    gen_num = int(row['gen_num'])

    # 생성 시작
    for which in range(gen_num):
        print(which)
        
        emotion_num = len(emtion_list)
        data_row = math.ceil(which/emotion_num)
        
        # 사용할 인지왜곡 문장
        distorted_thought = meta_data['thought'].iloc[data_row]
        print(distorted_thought)

        # 위 인지왜곡 문장의 class
        distorted_class = meta_data['pattern'].iloc[data_row]
        print(distorted_class)
        
        # 위 인지왜곡 class에 해당하는 정의
        definition = definition_dict.get(distorted_class)
        
        # 다양성을 주기 위한 감정 추가
        emotion = emtion_list[which % emotion_num]
    
        # 예시 추가
        # 추가할 예시 문장 개수
        prompt_num = int(row['example_num'])
        
        filtered_data = benchmark_data[benchmark_data['Dominant Distortion'] == distorted_class]['Patient Question'].dropna().tolist()
        
        # 랜덤하게 num_samples 개수만큼 선택
        samples = random.sample(filtered_data, min(prompt_num, len(filtered_data)))
        
        for i, sample in enumerate(samples, 1):
            prompt += f"\nExample {i}: {sample}"
                        
        prompt = prompt.format(emotion=emotion, sentence=distorted_thought, definition=definition)
        print(f"Prompt: {prompt}")
        
        try:
            start_time = time.time() # 시작 시간 기록
            
            # 입력을 토큰화하고 attention_mask 추가
            inputs = tokenizer(prompt, return_tensors="pt").to(device)
                            
            # 결과 생성
            with torch.no_grad():
                result = model.generate(
                    inputs['input_ids'], 
                    attention_mask=inputs['attention_mask'],  # attention_mask 추가
                    pad_token_id=model.config.pad_token_id,  # pad_token_id 추가
                    **params
                )
                
            # 생성된 텍스트 디코딩
            result = tokenizer.decode(result[0], skip_special_tokens=True)
            end_time = time.time() # 종료 시간 기록
            
            row['prompt'] = prompt  # 생성된 프롬프트 저장
            row['result'] = result  # 생성된 결과 저장
            row['time'] = round(end_time - start_time, 0) # 실행 시간 저장
            
            resultsheet.append_row(row.tolist(), table_range='A1') # Google 스프레드시트에 새로운 행 추가
            
            gc.collect()
            torch.cuda.empty_cache()
        except Exception as e:
            print(f"Error: {e}")
            resultsheet.append_row(row.tolist(), table_range='A1') # Google 스프레드시트에 새로운 행 추가
        
# 결과 저장
# resultsheet.append_row(row.tolist(), table_range='A1') # Google 스프레드시트에 새로운 행 추가
    

model_name                                     Qwen/Qwen2.5-7B-Instruct
max_new_tokens                                                     1024
temperature                                                            
top_p                                                                  
top_k                                                                  
repetition_penalty                                                     
gen_num                                                               5
example_num                                                           1
prompt_01             I am a psychology counselor, and you are a cli...
prompt_02                                  Show your current <emotion>;
prompt_03             You should put <sentence> as they are. *Don't ...
prompt_04               Should be in length <between 4 to 6 sentences>;
prompt_05                    The has one major <cognitive distortion> ;
prompt_06             The cognitive distortion's definition is g

Loading checkpoint shards: 100%|██████████| 4/4 [00:15<00:00,  3.80s/it]


0
I like my cats. I think one day they will plot against me and eat me in my sleep.
Magnification
Prompt: I am a psychology counselor, and you are a client taking a therapy. Respond to "How are you doing for the last week?" considering following requirements:
Show your current <emotion>;
You should put <sentence> as they are. *Don't change any words.;
Should be in length <between 4 to 6 sentences>;
The has one major <cognitive distortion> ;
The cognitive distortion's definition is given in <definition> ;
Your speech should contain only one <cognitive distortion>.
<emotion> : Depression
<sentence> : I like my cats. I think one day they will plot against me and eat me in my sleep.
<definition> : Exaggerating or Catastrophizing the outcome of certain events or behavior.
Here is an example:
Example 1: The inability to finish a task has got to be the one aspect of my personality I am simply failing to cope with. Through the years I have been able to learn and adapt myself so I can function 

c:\Users\saink\py_purpose\hugginhface\.venv\Lib\site-packages\transformers\models\qwen2\modeling_qwen2.py:623: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


1
I'm a vegan, and the restaurant served me a dish with fish in it. They're trying to kill me.
Overgeneralization
Prompt: I am a psychology counselor, and you are a client taking a therapy. Respond to "How are you doing for the last week?" considering following requirements:
Show your current <emotion>;
You should put <sentence> as they are. *Don't change any words.;
Should be in length <between 4 to 6 sentences>;
The has one major <cognitive distortion> ;
The cognitive distortion's definition is given in <definition> ;
Your speech should contain only one <cognitive distortion>.
<emotion> : Depression
<sentence> : I like my cats. I think one day they will plot against me and eat me in my sleep.
<definition> : Exaggerating or Catastrophizing the outcome of certain events or behavior.
Here is an example:
Example 1: The inability to finish a task has got to be the one aspect of my personality I am simply failing to cope with. Through the years I have been able to learn and adapt myself so

KeyboardInterrupt: 

In [27]:
# samples = random.sample(filtered_data, min(prompt_num, len(filtered_data)))
        
# for i, sample in enumerate(samples, 1):
#     prompt += f"\nExample {i}: {sample}"
#     # print('Example', i, ':', sample)
# print(prompt)

for index, row in df.iterrows():
    print(row)
    
    # 프롬프트 생성
    # prompt_01부터 prompt_12까지의 컬럼 값을 줄바꿈 형식으로 합치기
    raw_prompt = '\n'.join(row[f'prompt_{i:02d}'] for i in range(1, 13) if df.iloc[0][f'prompt_{i:02d}'])

    break
    
prompt = raw_prompt
for i, sample in enumerate(samples, 1):
    # prompt = raw_prompt + f"\nExample {i}: {sample}"
    prompt += f"\nExample {i}: {sample}"
print(prompt)

model_name                                     Qwen/Qwen2.5-7B-Instruct
max_new_tokens                                                     1024
temperature                                                            
top_p                                                                  
top_k                                                                  
repetition_penalty                                                     
gen_num                                                               5
example_num                                                           1
prompt_01             I am a psychology counselor, and you are a cli...
prompt_02                                  Show your current <emotion>;
prompt_03             You should put <sentence> as they are. *Don't ...
prompt_04               Should be in length <between 4 to 6 sentences>;
prompt_05                    The has one major <cognitive distortion> ;
prompt_06             The cognitive distortion's definition is g